In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow import keras

In [9]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [10]:
input_shape = (256, 256, 3)

In [13]:
model = keras.models.load_model('Inception_Resnet_SVM_Model\Inception_Resnet_svm_model.h5', compile=False)

In [14]:
path = 'Weights_Inception_Resnet_SVM/model_weights'
model.load_weights(path)

In [15]:
def predict(img_path):
    img = image.load_img(img_path, target_size=(256, 256))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.image.resize(x, (256,256))
    x /= 255

    res = model.predict(x)
    
    if(res[0][0]<0):
        return "GAN Fake"
    else:
        return "Real"

In [17]:
path = 'Test/GAN Fake/image20.jfif'
predict(path)

'GAN Fake'